In [14]:
import matplotlib as mpl
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sklearn.linear_model as linear_model
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from datetime import datetime
import plotly.express as px


In [6]:
from fredapi import Fred
api_fred = 'caf2a437b55be8f56406870c1bed3521'
fred = Fred(api_key= api_fred)
import quandl
quandl.api_config = 'J_fXGeVW_zC6RaDeJSQv'


In [27]:
class Contract:

    def __init__(self,S0,K,R,Sigma,T):
        """
        Class defines GBM contract
        S0: Spot price at time 0
        K: Strike price
        R: risk-free interest rate
        Sigma: annualized volatility of the option
        T: Time to maturity

        """
        self.S0 = S0
        self.K = K
        self.R = R
        self.Sigma = Sigma
        self.T = T
    def update_T(self,new_T:float):

        self.T = new_T
    def update_spot(self, Spot:float):
        self.S0 = Spot
    def update_rf(self, rf:float):
        self.R = rf
    def update_iv(self, IV:float):
        self.Sigma = IV
class CALL(Contract):

    def __init__(self,S0,K,R,Sigma,T):

        Contract.__init__(self,S0,K,R,Sigma,T)
        
    def update_T(self,new_T:float):

        self.T = new_T
    def update_spot(self, Spot:float):
        self.S0 = Spot
    def update_rf(self, rf:float):
        self.R = rf
    def update_iv(self, IV:float):
        self.Sigma = IV

class PUT(Contract):

    def __init__(self,S0,K,R,Sigma,T):

        Contract.__init__(self,S0,K,R,Sigma,T)
    def update_T(self,new_T:float):

        self.T = new_T
    def update_spot(self, Spot:float):
        self.S0 = Spot
    def update_rf(self, rf:float):
        self.R = rf
    def update_iv(self, IV:float):
        self.Sigma = IV

class Option_Pricer:

    def __init__(self,contract):

        """
        contract: use the CALL or PUT derived classes to defined the contract

        """

        self.contract = contract
        self.S0, self.K, self.R, self.Sigma, self.T = contract.S0,contract.K,contract.R,contract.Sigma, contract.T
        self.Type = self.contract.__class__.__name__

    def d1(self):
        D1 = (np.log(self.S0/self.K)+(self.R+ .5*self.Sigma**2)*self.T)/(self.Sigma*np.sqrt(self.T))
        return D1
    def d2(self):
        d_1 = self.d1()
        D2 = d_1 - self.Sigma*np.sqrt(self.T) 
        return D2
    def pv(self):
        return self.K*np.exp(-self.R*self.T)

    def __str__(self):

        return (f"{self.Type} option contract with strike {self.K} and volatility {self.Sigma}.")
        
    def price_derivative(self):
        D1 = self.d1()
        D2 = self.d2()
        PV = self.pv()

        if self.Type == 'CALL':

            return self.S0*stats.norm.cdf(D1)- PV*stats.norm.cdf(D2)
        else:

            return PV*stats.norm.cdf(-D2)-self.S0*stats.norm.cdf(-D1)

    def plot_contract(self):
        prices = self.price_derivative()

        if len(prices) <= 1:
            raise Exception('Cannot Graph 1 Price.')
        DF = pd.DataFrame(prices, index = self.S0, columns = [f"{self.Type}(K = {self.K})"])
        DF.index.name = 'Spot Price'

        fig = px.line(DF).update_layout(yaxis_title = 'Derivative Price')
        fig.add_shape(type ='line',x0 = self.K,x1 = self.K,y0= np.where(self.Type == 'CALL',0,),y1=np.where(self.Type == "CALL",max(prices),0), line = dict(color = "red"),name = 'Strike')

        return fig

    def plot_delta(self):
        title = f"{self.Type}(K = {self.K}, T = {self.T},vol = {self.Sigma},r = {self.R})"
        Deltas = self.delta()
        DF = pd.DataFrame(Deltas,index = self.S0, columns = [f"{self.Type}(K = {self.K})"])
        DF.index.name = 'Spot Price'

        fig = px.line(DF,title = title).update_layout(yaxis_title = "Delta")
        fig.add_shape(type ='line',x0 = self.K,x1 = self.K,y0= np.where(self.Type == 'CALL',0,-100),y1=np.where(self.Type == "CALL",100,0), line = dict(color = "red"),name = 'Strike')

        return fig
    def plot_gamma(self):

        title = f"{self.Type}(K = {self.K}, T = {self.T},vol = {self.Sigma},r = {self.R})"
        gammas = self.gamma()
        DF = pd.DataFrame(gammas,index = self.S0, columns = [f"{self.Type}(K = {self.K})"])
        DF.index.name = 'Spot Price'
        gamma_max = max(gammas)
        fig = px.line(DF,title = title).update_layout(yaxis_title = "Gamma")
        fig.add_shape(type ='line',x0 = self.K,x1 = self.K,y0=0,y1=gamma_max+1, line = dict(color = "red"),name = 'Strike')


        return fig
    def plot_vega(self):
        title = f"{self.Type}(K = {self.K}, T = {self.T},vol = {self.Sigma},r = {self.R})"
        vegas = self.vega()
        DF = pd.DataFrame(vegas,index = self.S0, columns = [f"{self.Type}(K = {self.K})"])
        DF.index.name = 'Spot Price'
        vega_max = max(vegas)
        fig = px.line(DF,title = title).update_layout(yaxis_title = "Vega")
        fig = fig.add_shape(type ='line',x0 = self.K,x1 = self.K,y0=0,y1=vega_max+1, line = dict(color = "red"),name = 'Strike')

        return fig
    def delta(self):
        if self.Type == 'CALL':
            return stats.norm.cdf(self.d1())
        else:
            return -stats.norm.cdf(-self.d1())
    def gamma(self):

        return stats.norm.pdf(self.d1())/(self.S0*self.Sigma*np.sqrt(self.T))

    def vega(self):

        return self.S0*np.sqrt(self.T)*stats.norm.pdf(self.d1())
    def theta(self):
        if self.Type == 'CALL':
            return ((-self.S0*stats.norm.pdf(self.d1()) * self.Sigma)/(2*np.sqrt(self.T)) - self.R * self.K*np.exp(-self.R*self.T) * stats.norm.cdf(self.d2()))/365
        else:
            return ((-self.S0*stats.norm.pdf(self.d1()) * self.Sigma)/(2*np.sqrt(self.T)) +self.R * self.K*np.exp(-self.R*self.T) * stats.norm.cdf(-self.d2()))/365

            # Divide Theta by 252 trading days
    def return_values(self):

        price = self.price_derivative()
        delta =self.delta()
        gamma = self.gamma()
        vega = self.vega()
        theta = self.theta()
        vals = np.array([price,delta,gamma,vega,theta])
        df = pd.DataFrame([vals],columns = ['Price','Delta','Gamma','Vega','Theta'],index = [f'{self.Type}(K = {self.K})'])
        return df



In [12]:
start = '2015-01-01'
end = datetime.now()
onemonth = (pd.DataFrame(fred.get_series("DGS1MO",observation_start=start,observation_end=end),columns=['ONE_M'])*1e-2).dropna()
onemonth.tail(2)

,ONE_M
2023-12-19,0.0551
2023-12-20,0.0550


In [30]:
T = 0.001
rf = onemonth.iloc[-1]['ONE_M']
K = 195
iv = 0.17
S0 =np.linspace(190,205,num = 100)
applejanOB = PUT(S0 = S0, K = 195, R = rf , Sigma=iv, T = T)
applejan = Option_Pricer(contract=applejanOB)
applejan.plot_contract()
